In [3]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename to `file`
file = 'Все глобалы за существование ЧБ.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

In [92]:
# Print the sheet names
sheet_names_list = xl.sheet_names
df = None
df_current = None

for i in range(len(sheet_names_list)):
    # Load a sheet into a DataFrame by name: df1
    df = xl.parse(sheet_names_list[i])
    df = df.loc[:,['id','mesTExt','type','dialogid', 'Mdate','AgentEmail']]
    df = df.drop_duplicates()
    #df.head()

    df['Mdate'] = pd.to_datetime(df['Mdate'],format='%Y-%m-%d')
    df['Mdate'] = df['Mdate'].dt.date

    df_filter = df['type'].isin([1])
    df1 = df[df_filter]

    a = df1.groupby(['dialogid', 'Mdate'])['mesTExt'].transform(lambda answer : " ".join(str(x) for x in answer))
    df12= a.to_frame()

    df4 = pd.concat([df12, df1.loc[:,['dialogid','Mdate']]], axis=1)
    df4 = df4.drop_duplicates()

    df_filter = df['type'].isin([2])
    df5 = df[df_filter]
    df5 = df5.loc[:,['dialogid', 'Mdate','AgentEmail']]
    df5 = df5.drop_duplicates()

    df_req=pd.merge(df4, df5, on=['dialogid', 'Mdate'], how='inner')
    if i>0:
        df = pd.concat([df_req, df_current], axis=0, ignore_index=True)
    df_current = df_req
    
df.to_csv("Заявки в чатбот.csv", encoding = 'utf-16', sep=",", index=False)   


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6641 entries, 0 to 6640
Data columns (total 4 columns):
mesTExt       6641 non-null object
dialogid      6641 non-null int64
Mdate         6641 non-null object
AgentEmail    6641 non-null object
dtypes: int64(1), object(3)
memory usage: 207.6+ KB


In [95]:
df.tail()

,mesTExt,dialogid,Mdate,AgentEmail
6636,Не получается зайти в ERP nan Сейчас попробую,12961952,2020-03-31,c2d_erp_tp@dv-nevada.ru
6637,Доброе утро!\nСейчас проверю,13494671,2020-03-31,c2d_centoffice_sa@dv-nevada.ru
6638,"Добрый день.\nне работает директ аксесс. Работаю на удаленке. Нет подключения, только к сети интернет",9863737,2020-03-31,briazgunov.d@dv-nevada.ru
6639,"Добрый день.\nне работает директ аксесс. Работаю на удаленке. Нет подключения, только к сети интернет",9863737,2020-03-31,c2d_georg_shkot_sa@dv-nevada.ru
6640,Добрый день! Не могу зайти в папку OBMEN (co-ok-3). Нужна срочно,12450202,2020-03-31,briazgunov.d@dv-nevada.ru
